This notebook contains scripts for both calculating kinematic data and visualizing it.
Input files must be `.csv` files generated by `rtklib-pos-to-csv.ipynb`.

In [3]:
import csv
import datetime as dt
import math
import geopy

In [ ]:
TIMESTAMP_FORMAT = "%Y/%m/%d %H:%M:%S.%f"

INPUT_CSV_HEADER = ["timestamp", "latitude", "longitude", "altitude", "fix-type", "satellites", "sdn",  "sde",  "sdu", "sdne", "sdeu", "sdun", "age", "ratio"]
OUTPUT_CSV_HEADER = [
    "timestamp",
    "fix-type",
    "latitude",
    "longitude",
    "ground-speed-raw", # mph
    "ground-speed-processed", # mph, TODO: should be smoothed
    "altitude", # meters
    "horizontal_accuracy",
    "vertical_accuracy",
    "speed_accuracy"
 ]

INPUT_CSV_FILEPATH = ""

In [ ]:
MIN_SATELLITES = 2

In [ ]:
lastPos = None
lastVel = None
lastTs = None

with open(INPUT_CSV_FILEPATH, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        if int(row["satellites"]) < MIN_SATELLITES: continue

        timestamp = dt.datetime.strptime(row["timestamp"], TIMESTAMP_FORMAT)
        lat = float(row["latitude"])
        lon = float(row["longitude"])
        alt = float(row["altitude"])

        sdn = float(row["sdn"])
        sde = float(row["sde"])
        sdu = float(row["sdu"])

        horizAcc = round(math.sqrt(sdn**2 + sde**2), 4) # TODO this is not statistically accurate
        vertAcc = round(sdu, 4) # TODO this may or may not need some math done on it

        # TODO fit some kind of curve to compute a smooth measurement of speed
        if lastPos:
            diff = geopy.distance.geodesic((last_pos[0], last_pos[1]), (lat, lng)).meters

            timediff = (ts - last_pos[2])
            groundSpeedRaw = round((diff/timediff)*2.23694, 4) # convert to mph

            if lastVel:
                acceleration = (groundSpeedRaw - lastVel[0]) / (ts - lastVel[1])   
            else:
                acceleration = 0

            last = groundSpeedRaw

            if acceleration > 100 or groundSpeedRaw > 45:
                continue

            speed_acc = 0 # TODO
        else:
            last_pos = (lat, lng, ts)
            continue

        last_pos = (lat, lng, ts)

